<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.30.2
    Uninstalling protobuf-6.30.2:


      Successfully uninstalled protobuf-6.30.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.61
    Uninstalling yfinance-0.2.61:


      Successfully uninstalled yfinance-0.2.61
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-05-14 11:03:09
-------------------
qualified stocks: 94
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.14 C
CY Investment:  1.41 C
Reserve:  4.03 L
Current:  1.27 C
-------------------
Today PnL: 74.16 K (0.59%)
Current PnL: -18.16 L (-12.88%)
CY Booked + Current PnL: -15.29 L (-10.85%)
-------------------
Total profit:  3.94 L
Total loss:  -22.11 L
-------------------
Total Booked + Current PnL: 11.69 L (10.25%)
Total Booked PnL: 29.85 L (26.17%)
Curr Year Booked PnL: 2.87 L (2.25%)
Prev Year Booked PnL: 26.98 L (23.66%)
Est FTT:  2.08 C
Est FTT PnL: 80.04 L (62.78%)
-------------------
Est LTT:  2.69 C
Est LTT PnL: 1.42 C (111.13%)
Deployed:  1.14 C
Current:  1.27 C
CAGR/XIRR %: 8.08%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MRF,114777.60,150504.14,14.58,X-MC,89.36,140610.0,25832.0,9899.0,0.13,22.51,7.04,31.13,215.0,2.61,1.14,36.97,AR,ATH,TYRES
56,PIDILITIND,2504.06,3576.00,-15.79,X-LC,47.87,93300.0,18178.0,13976.0,0.30,24.20,14.98,42.81,87.0,1.30,0.76,17.01,X40,BTT,CHEMICALS
69,SBILIFE,1496.49,1924.99,-1.16,H-LC,80.85,146899.0,21194.0,14793.0,0.51,16.86,10.07,28.63,116.0,1.43,1.19,30.87,AR,ATH,INSURANCE
70,SBIN,760.30,863.00,-12.51,M-LC,55.32,213259.0,11780.0,15440.0,0.37,5.85,7.24,13.51,91.0,0.76,1.73,16.83,XY25,NTT,BANKS
25,DMART,3484.00,5112.00,-21.32,X-LC,10.64,96917.0,13301.0,25770.0,-0.64,15.91,26.59,46.73,123.0,0.52,0.78,18.63,AR,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
27,FINEORG,4771.46,7168.71,-20.50,H-SC,41.49,75733.0,-5382.0,46137.0,3.65,-6.63,60.92,50.24,128.0,-0.12,0.61,27.17,X40N,ATH,CHEMICALS
65,SAIL,130.64,228.00,82.04,M-MC,60.64,206916.0,-18046.0,185707.0,3.59,-8.02,89.75,74.53,217.0,-0.10,1.68,20.14,XY24,BTT,STEEL
6,ANGELONE,2328.67,3033.00,2.48,H-SC,76.60,167180.0,15816.0,29959.0,3.39,10.45,17.92,30.25,131.0,0.53,1.35,32.32,X40N,NTT,FINANCE
73,SIEMENS,4406.90,7969.85,-16.92,X-LC,56.91,75135.0,-35037.0,124108.0,3.04,-31.80,165.18,80.85,10.0,-0.28,0.61,10.66,AR,ATH,ELECTRICAL
75,SONACOMS,578.05,1006.00,-29.20,M-MC,86.17,91630.0,-9529.0,84419.0,3.04,-9.42,92.13,74.03,21.0,-0.11,0.74,28.78,AR,BTT,AUTO


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
90,VIPIND,488.80,718.00,-785.50,H-SC,97.87,65305.0,-30011.0,74702.0,-3.53,-31.49,114.39,46.89,51.0,-0.40,0.53,28.81,X40N,NTT,MISC
81,TCS,3928.29,4998.00,-15.84,X-LC,74.47,189810.0,-22318.0,80081.0,-2.91,-10.52,42.19,27.23,8.0,-0.28,1.54,8.77,X40,BTT,IT
37,INDIAMART,2311.97,4911.36,-53.15,H-SC,68.09,101490.0,-237.0,114613.0,-1.86,-0.23,112.93,112.43,NaN,-0.00,0.82,19.78,AR,ATH,MISC
7,ASIANPAINT,2961.56,4250.00,-21.56,X-LC,5.32,155992.0,-45394.0,133014.0,-1.32,-22.54,85.27,43.51,113.0,-0.34,1.26,7.59,X40,BTT,PAINTS
21,COLPAL,2837.00,3808.06,5.16,X-MC,39.36,128675.0,-13175.0,61725.0,-0.98,-9.29,47.97,34.23,32.0,-0.21,1.04,9.49,X40,ATH,FMCG


In [12]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,TATAMOTORS,776.02,1065.00,-53.87,M-LC,82.98,185911.0,-19734.0,96320.0,-0.87,-9.60,51.81,37.24,1.0,-0.20,1.51,21.01,XY24,NTT,AUTO
2,ABB,5319.37,8953.12,-37.83,H-LC,73.40,142175.0,9191.0,81651.0,0.90,6.91,57.43,68.31,2.0,0.11,1.15,15.93,AR,ATH,ELECTRICAL
24,DLF,683.35,1031.70,-24.28,H-LC,54.26,125541.0,2538.0,60159.0,2.89,2.06,47.92,50.98,3.0,0.04,1.02,14.70,AR,ATH,REALTY
10,AWL,342.88,485.00,-68.61,H-MC,9.57,170021.0,-47708.0,137955.0,1.77,-21.91,81.14,41.45,5.0,-0.35,1.38,13.57,XY24,NTT,FMCG
92,WHIRLPOOL,1151.42,2270.00,-35.42,M-SC,92.55,162547.0,16317.0,125746.0,1.10,11.16,77.36,97.15,7.0,0.13,1.32,38.86,X40,NTT,DURABLES


In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MRF,114777.60,150504.14,14.58,X-MC,89.36,140610.0,25832.0,9899.0,0.13,22.51,7.04,31.13,215.0,2.61,1.14,36.97,AR,ATH,TYRES
52,MEDANTA,1022.60,1486.00,6.76,X-MC,13.83,188085.0,32650.0,37786.0,-0.34,21.01,20.09,45.32,198.0,0.86,1.52,27.98,XY24,NTT,HEALTHCARE
56,PIDILITIND,2504.06,3576.00,-15.79,X-LC,47.87,93300.0,18178.0,13976.0,0.30,24.20,14.98,42.81,87.0,1.30,0.76,17.01,X40,BTT,CHEMICALS
4,ADANIPORTS,1103.69,1583.00,-2.58,M-LC,88.30,188224.0,37018.0,28648.0,0.34,24.48,15.22,43.43,37.0,1.29,1.52,30.47,XY24,NTT,MISC


In [14]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,LUXIND,1448.19,2442.00,33.16,M-SC,29.79,138206.0,628.0,93787.0,1.95,0.46,67.86,68.62,55.0,0.01,1.12,19.82,X40N,NTT,TEXTILES
57,POLYCAB,5005.92,7541.35,6.49,H-MC,87.23,166670.0,26504.0,44484.0,-0.92,18.91,26.69,50.65,54.0,0.60,1.35,27.20,X40N,ATH,CABLES
6,ANGELONE,2328.67,3033.00,2.48,H-SC,76.60,167180.0,15816.0,29959.0,3.39,10.45,17.92,30.25,131.0,0.53,1.35,32.32,X40N,NTT,FINANCE
4,ADANIPORTS,1103.69,1583.00,-2.58,M-LC,88.30,188224.0,37018.0,28648.0,0.34,24.48,15.22,43.43,37.0,1.29,1.52,30.47,XY24,NTT,MISC
93,WIPRO,243.46,420.00,-8.80,M-LC,44.68,156271.0,5326.0,104123.0,0.19,3.53,66.63,72.51,58.0,0.05,1.27,17.66,XR,NTT,IT


In [15]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
93,WIPRO,243.46,420.00,-8.80,M-LC,44.68,156271.0,5326.0,104123.0,0.19,3.53,66.63,72.51,58.0,0.05,1.27,17.66,XR,NTT,IT
12,BANKINDIA,113.49,190.00,-34.31,M-MC,37.23,129732.0,580.0,86492.0,1.28,0.45,66.67,67.42,71.0,0.01,1.05,26.18,XR,NTT,BANKS
37,INDIAMART,2311.97,4911.36,-53.15,H-SC,68.09,101490.0,-237.0,114613.0,-1.86,-0.23,112.93,112.43,NaN,-0.00,0.82,19.78,AR,ATH,MISC
18,CAMS,3643.00,5250.99,-9.33,H-SC,6.38,101984.0,-20.0,45046.0,0.44,-0.02,44.17,44.14,47.0,-0.00,0.83,16.22,XR,ATH,MISC
91,VOLTAS,1272.00,1929.20,-30.74,H-MC,14.89,138347.0,-1573.0,73863.0,-0.63,-1.12,53.39,51.67,19.0,-0.02,1.12,4.49,AR,ATH,AC


In [16]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,UJJIVANSFB,52.77,60.00,37.69,M-SC,77.66,119205.0,-23274.0,42795.0,0.41,-16.34,35.90,13.70,219.0,-0.54,0.97,40.69,X40N,NTT,BANKS
53,MRF,114777.60,150504.14,14.58,X-MC,89.36,140610.0,25832.0,9899.0,0.13,22.51,7.04,31.13,215.0,2.61,1.14,36.97,AR,ATH,TYRES
69,SBILIFE,1496.49,1924.99,-1.16,H-LC,80.85,146899.0,21194.0,14793.0,0.51,16.86,10.07,28.63,116.0,1.43,1.19,30.87,AR,ATH,INSURANCE
86,UNIONBANK,123.87,163.00,-12.58,M-LC,75.53,152870.0,12030.0,32454.0,2.81,8.54,21.23,31.59,83.0,0.37,1.24,32.79,XY24,NTT,BANKS
52,MEDANTA,1022.60,1486.00,6.76,X-MC,13.83,188085.0,32650.0,37786.0,-0.34,21.01,20.09,45.32,198.0,0.86,1.52,27.98,XY24,NTT,HEALTHCARE


In [17]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
58,QUESS,711.66,986.00,NaN,NaN,100.00,81161.0,-74693.0,134776.0,1.98,-47.92,166.06,38.55,NaN,-0.55,0.66,24.28,XY24,NTT,MISC
79,TATAELXSI,7332.28,9000.00,-18.67,X-MC,98.94,85316.0,-17336.0,40687.0,1.80,-16.89,47.69,22.74,33.0,-0.43,0.69,28.24,X40N,NTT,IT
90,VIPIND,488.80,718.00,-785.50,H-SC,97.87,65305.0,-30011.0,74702.0,-3.53,-31.49,114.39,46.89,51.0,-0.40,0.53,28.81,X40N,NTT,MISC
1,AARTIIND,487.04,919.00,35.69,M-SC,96.81,77947.0,-3389.0,75523.0,0.01,-4.17,96.89,88.69,118.0,-0.04,0.63,31.44,XR,NTT,CHEMICALS
78,TANLA,917.30,1963.11,-40.61,H-SC,95.74,145599.0,-93816.0,366778.0,0.09,-39.19,251.91,114.01,29.0,-0.26,1.18,33.54,AR,ATH,IT


In [18]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.29
1,25,40.99
2,50,69.52


In [19]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    34.86
X40     14.45
X40N    14.05
AR      13.84
XR      10.33
XY25    10.23
X200     1.20
SR       1.10
Name: CurrAlloc%, dtype: float64

In [20]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    22.33
X-LC    18.21
M-SC    16.65
M-LC     8.43
H-MC     7.82
X-MC     6.76
H-LC     5.70
M-MC     5.56
X-SC     2.80
L-SC     2.26
Name: CurrAlloc%, dtype: float64

In [21]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,11.75,-2.30,34.85
BANKS,11.13,-9.76,63.59
IT,9.29,-20.96,88.44
FINANCE,7.51,-19.26,73.07
MISC,7.19,-27.69,82.63
INSURANCE,6.10,-2.60,33.54
PAINTS,5.54,-20.03,55.55
ELECTRICAL,4.14,-8.80,64.24
REFINERIES,3.65,9.56,7.10


In [22]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3048421.0,27
XR,1270505.0,15
AR,1248097.0,15
X40N,1196680.0,17
X40,812452.0,12
SR,209048.0,2
XY25,148907.0,5
X200,70280.0,1


In [23]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2432366.0,22
M-SC,1952715.0,20
X-LC,736553.0,12
M-MC,582431.0,5
H-MC,430914.0,7
X-SC,362665.0,4
M-LC,340082.0,6
X-MC,319861.0,6
L-SC,318293.0,4


In [24]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      930146.0      6
M-SC       XY24      770101.0      6
H-SC       X40N      554929.0      7
           AR        543078.0      3
M-SC       XR        431561.0      5
M-MC       XY24      411520.0      3
X-LC       X40       378809.0      5
M-SC       X40N      372026.0      5
H-SC       XR        303793.0      4
M-LC       XY24      220519.0      4
M-SC       X40       199022.0      2
H-LC       AR        188994.0      4
X-LC       AR        187981.0      3
H-MC       XY24      181384.0      2
L-SC       XR        164537.0      2
X-MC       XY24      141064.0      2
X-SC       SR        123305.0      1
           XR        121292.0      1
           X40N      118068.0      2
X-MC       X40N      107173.0      2
M-LC       XR        104123.0      1
H-SC       X40       100420.0      2
X-LC       XY25       99483.0      3
M-SC       AR         94262.0      1
L-SC       XY24       88155.0      1
M-MC       XR         86492.0      1
M-SC       SR         85743.0      1
M-MC       AR         84419.0      1
H-MC       AR         73863.0      1
           X40        72476.0      2
X-LC       X200       70280.0      1
L-SC       AR         65601.0      1
X-MC       X40        61725.0      1
H-MC       XR         58707.0      1
           X40N       44484.0      1
H-LC       XY25       33984.0      1
M-LC       XY25       15440.0      1
X-MC       AR          9899.0      1

In [25]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 37.0 seconds
